A look at how the test images actually look

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

In [ ]:
import pandas as pd
import numpy as np
import torch


INPUT_PATH = '/kaggle/input/bengaliai-cv19'
test_df = pd.read_csv(INPUT_PATH + '/test.csv')
submission_df = pd.read_csv(INPUT_PATH + '/sample_submission.csv')

results = []

In [ ]:
import cv2
from tqdm.auto import tqdm

HEIGHT = 137
WIDTH = 236
SIZE = 128

def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax

def crop_resize(img0, size=SIZE, pad=16):
    #crop a box around pixels large than the threshold 
    #some images contain line at the sides
    ymin,ymax,xmin,xmax = bbox(img0[5:-5,5:-5] > 80)
    #cropping may cut too much, so we need to add it back
    xmin = xmin - 13 if (xmin > 13) else 0
    ymin = ymin - 10 if (ymin > 10) else 0
    xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
    ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
    img = img0[ymin:ymax,xmin:xmax]
    #remove lo intensity pixels as noise
    img[img < 28] = 0
    lx, ly = xmax-xmin,ymax-ymin
    l = max(lx,ly) + pad
    #make sure that the aspect ratio is kept in rescaling
    img = np.pad(img, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')
    return cv2.resize(img,(size,size))

def Resize(df,size=128):
    resized = {} 
    df = df.set_index('image_id')
    for i in tqdm(range(df.shape[0])):
       # image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size))
        image0 = 255 - df.loc[df.index[i]].values.reshape(137,236).astype(np.uint8)
        #normalize each image by its max val
        img = (image0*(255.0/image0.max())).astype(np.uint8)
        image = crop_resize(img)
        resized[df.index[i]] = image.reshape(-1)
    resized = pd.DataFrame(resized).T.reset_index()
    resized.columns = resized.columns.astype(str)
    resized.rename(columns={'index':'image_id'},inplace=True)
    return resized

In [ ]:
from torch.utils.data import Dataset, DataLoader

h, w = 128, 128
# h, w = 137, 236
class GraphemeDataset(Dataset):
    def __init__(self,df,_type='train'):
        self.df = df
        self.data = df.iloc[:, 1:].values
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        name = self.df.iloc[idx,0]
        image = self.data[idx, :].reshape(h, w).astype(np.float)
        image = np.stack([image, image, image], axis=-1)
        image = np.transpose(image, (2, 0, 1))
        return image,name

In [ ]:
test = pd.read_csv('/kaggle/input/bengaliai-cv19/test.csv')
test_data = ['test_image_data_0.parquet','test_image_data_1.parquet','test_image_data_2.parquet','test_image_data_3.parquet']

In [ ]:
%%time
## Inference a little faster using @Iafoss and  @peters technique
row_id, target = [], []
imgs = []
for fname in test_data:
    print(fname)
    data = pd.read_parquet(f'/kaggle/input/bengaliai-cv19/{fname}')
    data = Resize(data)
    test_image = GraphemeDataset(data)
    dl = torch.utils.data.DataLoader(test_image, batch_size=128, num_workers=4, shuffle=False)
    with torch.no_grad():
        for x, y in tqdm(dl):
            # x = x.unsqueeze(1).float().cpu()
            x = x.squeeze(1).detach().cpu().numpy()
            for idx in range(x.shape[0]):
                cimg = np.transpose(x[idx, :, :, :], (1, 2, 0))
                imgs.append(cimg)           

In [ ]:
fig, axs = plt.subplots(4, 3, figsize=(20,10))

for row in range(4):
    for col in range(3):
        idx = row*3 + col
        img = imgs[idx]
        axs[row, col].imshow(img, cmap='gray')
        axs[row, col].axis('off')